<a href="https://colab.research.google.com/github/dlagur/ProjectHanium/blob/main/KoBERT_%EC%9E%90%EA%B8%B0%EC%86%8C%EA%B0%9C%EC%84%9C_%EC%A7%81%EB%AC%B4%ED%8C%90%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 데이터 불러오기

In [1]:
import pandas as pd

data = pd.read_csv('/content/RawData.csv')

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
# Transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용
# device = torch.device('cuda:0')

In [6]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-07-18 06:23:52--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.148.67
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.148.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  6.70MB/s    in 2.1s    

2022-07-18 06:23:55 (6.70 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-07-18 06:23:55--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.56.86
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.56.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.

## 전처리

In [8]:
dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [9]:
data

,question,answer,label
0,"[질문 Q1.NC 인턴십에 지원하게 된 배경과 인턴십을 통해 경험해보고 싶은 업무,...",[학부연구생을 할 당시 연구실 홈페이지를 맡아 개발한 경험이 있으며 스프링부트와 템...,0
1,"[질문 Q1.자신에 대해 자유롭게 표현해 주세요. 보기, 질문 Q2.희망하는 지원 ...","[""Agile &amp; Active""새로운 것을 배우는 데 적극적입니다. 학과 수...",0
2,[질문 Q1.본인에 대해 자유롭게 서술해주세요. 보기],"[""욕심에 더해 애착까지 가졌다면 이루지 못할 것은 없다""저를 한 단어로 소개한다면...",0
3,[질문 Q1.MyCompetency 지원 분야와 관련된 구체적인 지식이나 경험을 기...,[저는 학부 연구생을 하면서 연구실 홈페이지 개발 및 관리를 맡은 경험이 있습니다....,0
4,[질문 Q1.현대백화점그룹이 지향하는 4가지 실천가치(열정/소통/창의/파트너십 중 ...,[열정 - 목표를 정하고 이를 달성하기 위해 몰입하는 과정을 통해 성과를 낸 경험이...,0
...,...,...,...
1787,"[질문 Q1.자유롭게 자신에 대해 소개해주세요. 보기, 질문 Q2. 당사에 지원하게...","[""목표로 가는 단계의 중요성""단계를 중시하는 삶이 지금까지 저의 인생에서 가장 강...",8
1788,[질문 Q1.CJ제일제당과 해당 직무에 지원한 동기는 무엇인가요?① CJ제일제당이어...,"[""VIPS에서 제일제당으로"" 7개월 동안 VIPS OOOOOO에서 Floor 업무...",8
1789,"[질문 Q1. 직무 관련 역량(본인만의 경쟁력) 보기, 질문 Q2. 직무 관련 보기]","[""끈기 : 목표를 잡으면 반드시 해냅니다.""고등학교를 졸업하고 ‘장기여행’을 하고...",8
1790,"[질문 Q1.성장 과 정보기, 질문 Q2.지 원동기 및 포부보기]",[고등학교까지는 그저 시키는 것만 하던 평범한 학생이었습니다. 시키는 것도 제대로 ...,8


In [10]:

data_list = []
for q, label in zip(data['answer'], data['label'])  :
    pro_data = []
    pro_data.append(q)
    pro_data.append(str(label))

    data_list.append(pro_data)

In [11]:
print(data_list[0])
print(data_list[-1])

["[학부연구생을 할 당시 연구실 홈페이지를 맡아 개발한 경험이 있으며 스프링부트와 템플릿 엔진을 사용하여 백오피스 기능을 개발하고 온프레미스 환경에 배포했습니다. 이러한 경험을 통해 백엔드에 대한 흥미와 관심이 커졌고 이후 졸업 프로젝트에서는 얼굴인식을 사용한 서비스에서 웹 백엔드를 맡아 개발했습니다. 그리고 현재는 다양한 아키텍처와 기술에 관심을 갖고 MSA기반 'OOO'이라는 토이 프로젝트를 진행하고 있습니다.프로젝트를 진행하고 다양한 기술을 접하고 관심을 갖게 되면서 더 다양한 경험을 해보고 싶었습니다. 혼자 하거나 작은 규모의 프로젝트에서 사용하거나 도입할 수 있는 기술에는 한계가 있습니다. 그래서 저는 인턴십에 지원했습니다. 인턴십을 통해 어떠한 방식으로 MSA가 실현되고 있는지, 로그는 어떤 방식으로 수집하는지, 배포 자동화는 어떤 식으로 하는 지 등 다양한 부분에서 고민했던 것과 비교해보며 배우고 싶습니다.                        글자수 466자806Byte                    , 'OOO'이라는 토이 프로젝트(2인)를 진행하고 있습니다. 해당 프로젝트는 MSA 기반으로 설계했습니다. 도메인을 고려하여 5개의 마이크로서비스로 쪼개 설계하고 모델링 했습니다. 마이크로서비스 사이의 효율적인 통신을 위해 gRPC를 도입하게 되었으며 캐싱을 위한 redis를 사용하는 등의 새로운 기술을 공부하며 도입했습니다. 새로운 기술을 도입하려면 스프링부트에서 의존성이나 컨피그 설정을 해야 할 일이 많았습니다. 그에 따라 스프링 독스를 다시 보면 기존에 부족했던 스프링에 대한 개념을 채우고 있습니다. SA 또한 공부하고 시작한 것이 아니라 간단한 개념만 파악하고 시작했기 때문에 책을 읽으며 공부를 하고 있습니다. 프로젝트를 진행하면서 부족한 부분을 많이 느끼게 되어 다시 근간이 되는 부분과 '왜'에 대한 고민을 하고 공부하고 있습니다.                        글자수 418자712Byte                 

## Train & Test Data

In [12]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))

1344


In [13]:
print(len(dataset_train)) # 1344 (28945) 
print(len(dataset_test)) # 448 (9649)

1344
448


## KoBERT 입력 데이터로 만들기

* 데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 진행

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair = pair) # 패딩과 토큰화 진행
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## KoBERT 학습모델 만들기

In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
torch.cuda.is_available()

False

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5) #.to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## KoBERT 모델 학습시키기

In [21]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long()# .to(device)
        valid_length= valid_length
        label = label.long() #.to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long() # .to(device)
        valid_length= valid_length
        label = label.long()# .to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.3308258056640625 train acc 0.0625
epoch 1 train acc 0.11086309523809523


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.1875


  0%|          | 0/21 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.9775073528289795 train acc 0.203125
epoch 5 train acc 0.2700892857142857


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5 test acc 0.1875


## 새로운 문장 테스트

In [26]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_resume):

    data = [predict_resume, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long() # .to(device)

        valid_length= valid_length
        label = label.long() # .to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("IT")
            elif np.argmax(logits) == 1:
                test_eval.append("Construction")
            elif np.argmax(logits) == 2:
                test_eval.append("Management")
            elif np.argmax(logits) == 3:
                test_eval.append("Marketing")
            elif np.argmax(logits) == 4:
                test_eval.append("Distribution")
            elif np.argmax(logits) == 5:
                test_eval.append("Manufacturing")
            elif np.argmax(logits) == 6:
                test_eval.append("RnD")
            elif np.argmax(logits) == 7:
                test_eval.append("Sales_support")
            elif np.argmax(logits) == 8:
                test_eval.append("Sales")

        print(">> 입력하신 자기소개서는 " + test_eval[0] + " 직무로 판단됩니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


* 링커리어 자기소개서 스크랩해서 직무가 일치하는지 테스트

In [31]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    resume = input("자기소개서를 입력해주세요 : ")
    if resume == 'stop' :
        break
    predict(resume)
    print("\n")

자기소개서를 입력해주세요 : 올해 여름 진행했던 팀 프로젝트 경험에 대해 말씀드리겠습니다. 저는 웹 개발 공부를 하며 여러 개인 프로젝트를 수행하며 실력을 쌓아갔습니다. 하지만 혼자 강의를 통해 공부를 진행하다 보니 개인 프로젝트만 진행하였고 팀 프로젝트도 경험해보고 싶었습니다. 그래서 저는 프런트엔드와 백엔드를 구분해서 진행하는 프로젝트 진행을 위해 커뮤니티를 통해 팀을 구했습니다. 하지만 첫 팀 프로젝트 회의 날 저는 아무 말도 할 수 없었습니다. 지금까지 개인 프로젝트만 진행하다 보니 ERD 같은 기초적인 용어도 몰랐으며 Git 전략, slack, Notion 등 다양한 협업 툴에 대해서도 무지하였습니다 그래서 저는 첫 회의 이후 저는 회의에서 나온 내용을 모두 필기해놓고 하나하나 검색을 통해 공부하였습니다. 또한 코드를 짜는 방법에 있어 기존에 제가 짜던 방식과 다른 부분이 많아 어려움을 겪었습니다. 그래서 프로젝트를 진행하면서 모르는 부분 하나하나 다 검색을 통해 사용법을 알아봤고 개인 프로젝트에 먼저 적용해 보고 사용법을 익히고 팀 프로젝트 코드에 적용하며 어려운 부분에 있어서는 팀원분들의 도움을 받아서 프로젝트를 진행하였습니다. 결국 제가 맡은 부분의 도메인, 서비스, 컨트롤러, DAO, Test 코드를 프로젝트를 통해 배운 방식을 통해 구현할 수 있었고 그 결과 첫 팀 프로젝트를 성공적으로 마무리할 수 있었습니다.  


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 자기소개서는 IT 직무로 판단됩니다.


자기소개서를 입력해주세요 : stop
